In [3]:
import cv2
import os 
import numpy as  np 

In [ ]:
IMG_FILE_EXTS = ['.png', '.jpg', '.tif', '.tiff', '.bmp']

def load_dataset(img_dir, resize_shape=(224,224), num_classes=1, num_bboxes=2):
    # resize_shape = (width, height)
    # class info : num_bboxes * 5(= p, x, y, wi, h ) + num_classes
    length_cls_info = num_bboxes * 5 + num_classes

    x_train = [] 
    y_train = []

    # load file lists
    file_list = os.listdir(img_dir)
    img_file_list = [x for x in file_list if os.path.split(x)[1].lower() in IMG_FILE_EXTS]

    # check image file 과 label file 수 확인 

    total = len(img_file_list)
    x_train = np.zeros((total, resize_shape[1], resize_shape[0], length_cls_info), dtype='float32')    
    y_train = np.zeros((total, 7,7, length_cls_info), dtype='float32')    
    # load image files 
    for i, img_file in enumerate(img_file_list):
        # decompose filename and extention
        filename, _ = os.path.splitext(img_file)        
        label_file = filename + '.txt'
       
        # read image, resize and bgr->rgb
        img = resize_image(img_file, resize_shape)
        # load labels
        label_list = read_label_file(label_file)
    
        # convert labels -> yolo features = ?x7x7x(5xnum_bboxes + num_classes) 
        yolo_feature = np.zeros((7,7,length_cls_info), dtype='float32')
        yolo_feature = convert_bbox_info_to_yolo_feature(yolo_feature, label_list)

        x_train[i] = img
        y_train[i] = yolo_feature
        
    return x_train, y_train 

def read_label_file(label_file):
    labels = []

    return labels

def resize_image(filename, resize_shape=(224,224)):
    img = cv2.imread(filename).astype('float32') / 255.0
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, dsize=resize_shape)
    return img


def convert_bbox_info_to_yolo_feature(yolo_feature, label_list):

    for label in label_list:
        i_x = int(label[1] * 6)
        i_y = int(label[2] * 6)
        x = label[1] * 6 - i_x
        y = label[2] * 6- i_y
        w = label[3]
        h = label[4]
        class_id = label[0]
        bbox = [1., x, y, w, h]
        # bbox info
        yolo_feature[i_y,i_x,:5] = bbox
        # class info 
        yolo_feature[i_y,i_x,10 + class_id] = 1.

        return yolo_feature
    


In [28]:
rows, cols = 224, 244

label1 = np.array([50,20,40,100]) / 244.
print(label1)

out = np.zeros((7,7,30), dtype='float32')

[0.20491803 0.08196721 0.16393443 0.40983607]


In [30]:
num_classes = 2
num_bbox = 2

yolo = np.zeros((7,7, 5*num_bbox + num_classes), dtype='float32')
print('yolo shape:', yolo.shape)

# class, x, y, width, height
labels = [[0, 0.459016393442623, 0.459016393442623, 0.4098360655737705, 0.4098360655737705],
[1, 0.20491803, 0.08196721, 0.16393443 ,0.40983607]]

for label in labels:
    i_x = int(label[1] * 6)
    i_y = int(label[2] * 6)
    x = label[1] * 6 - i
    y = label[2] * 6- j
    w = label[3]
    h = label[4]
    class_id = label[0]
    bbox = [1, x, y, w, h]
    # bbox info
    yolo[i_y,i_x,:5] = bbox
    # class info 
    yolo[i_y,i_x,10 + class_id] = 1.
    print(i_y,i_x, x, y, w, h)

print(yolo)

yolo shape: (7, 7, 12)
2 2 1.754098360655738 2.754098360655738 0.4098360655737705 0.4098360655737705
0 1 0.22950818000000006 0.49180325999999996 0.16393443 0.40983607
[[[0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.        ]
  [1.         0.22950818 0.49180326 0.16393442 0.40983608 0.
   0.         0.         0.         0.         0.         1.        ]
  [0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         